<a href="https://colab.research.google.com/github/zhan0249/Embedding-Layer-Testing/blob/main/Testing_Embedding_Layer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is created to test Keras embedding layer performance between learnt embedding and pre-trained GloVe (100 dimension) embedding.

Althought pre-trained GloVe embedding performed must faster (just performing looking-up), the learnt embedding had a better performance using only 10 epochs.

# Download Glove Embedding

In [14]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!apt install unzip


--2024-09-01 04:38:52--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2024-09-01 04:38:52--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2024-09-01 04:38:52--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [18]:
!unzip "glove.6B.zip"

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [15]:
import os
os.listdir()

['.config',
 'glove.42B.300d.zip',
 'glove.42B.300d.txt',
 'glove.6B.zip',
 'sample_data']

In [38]:
!pip install nlppreprocess

In [42]:
import numpy as np
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.text import Tokenizer

from numpy import array
from numpy import asarray
from numpy import zeros

import pandas as pd
import re

from keras.utils import to_categorical
from nlppreprocess import NLP


In [19]:
embeddings_index = dict()
f = open('glove.6B.100d.txt')
for line in f:
 values = line.split()
 word = values[0]
 coefs = asarray(values[1:], dtype='float32')
 embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [62]:
# create a weight matrix for words in training docs
embedding_matrix = zeros((max_features, 100))
for word, i in t.word_index.items():
  embedding_vector = embeddings_index.get(word)
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector

In [72]:
embedding_matrix[:5]

array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0. 

# Import Data

In [ ]:
!wget !wget https://archive.ics.uci.edu/ml/machine-learning-databases/00462/drugsCom_raw.zip

In [21]:
from google.colab import drive
drive.mount('\content\drive')

TIMEOUT: <pexpect.popen_spawn.PopenSpawn object at 0x79d01203d3f0>
searcher: searcher_re:
    0: re.compile('google.colab.drive MOUNTED')
    1: re.compile('root@c9b7c01aa3bf-522be8a82a1a40c48d14e17ad6671d5c: ')
    2: re.compile('Drive File Stream encountered a problem and has stopped')
    3: re.compile('drive EXITED')
    4: re.compile('The domain policy has disabled Drive File Stream')
<pexpect.popen_spawn.PopenSpawn object at 0x79d01203d3f0>
searcher: searcher_re:
    0: re.compile('google.colab.drive MOUNTED')
    1: re.compile('root@c9b7c01aa3bf-522be8a82a1a40c48d14e17ad6671d5c: ')
    2: re.compile('Drive File Stream encountered a problem and has stopped')
    3: re.compile('drive EXITED')
    4: re.compile('The domain policy has disabled Drive File Stream')

## Process Data

In [22]:
!unzip '/content/contentdrive/MyDrive/Colab Data/drugsCom_raw.zip'

Archive:  /content/contentdrive/MyDrive/Colab Data/drugsCom_raw.zip
  inflating: drugsComTest_raw.tsv    
  inflating: drugsComTrain_raw.tsv   


In [26]:
df = pd.read_csv('drugsComTrain_raw.tsv', sep='\t', engine='python')
df = df[['review', 'rating']]
df.head()

,review,rating
0,"""It has no side effect, I take it in combinati...",9.0
1,"""My son is halfway through his fourth week of ...",8.0
2,"""I used to take another oral contraceptive, wh...",5.0
3,"""This is my first time using any form of birth...",8.0
4,"""Suboxone has completely turned my life around...",9.0


In [28]:
df.shape

(161297, 2)

In [29]:
df['rating'] = [1 if int(x)>5 else 0 for x in df['rating']]
df.head()

,review,rating
0,"""It has no side effect, I take it in combinati...",1
1,"""My son is halfway through his fourth week of ...",1
2,"""I used to take another oral contraceptive, wh...",0
3,"""This is my first time using any form of birth...",1
4,"""Suboxone has completely turned my life around...",1


In [123]:
Y=df['rating']

In [125]:
X=df['review']

In [124]:
type(Y)

pandas.core.series.Series

In [120]:
from sklearn.model_selection import train_test_split

In [126]:
X_train,X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.4)

In [129]:
type(X_train)

pandas.core.series.Series

In [134]:
X_train.shape

(96778,)

In [135]:
Y_train.shape

(96778,)

In [136]:
X_test.shape

(64519,)

# Use Pre-Trained GloVe Embedding

In [145]:

max_len = 40

tokenizer = Tokenizer()
tokenizer.fit_on_texts(list((X_train.values)))

list_tokenized_train = tokenizer.texts_to_sequences(X_train.values)
list_tokenized_test = tokenizer.texts_to_sequences(X_test.values)

X_t = pad_sequences(list_tokenized_train, maxlen=max_len,padding='post')
X_v = pad_sequences(list_tokenized_test, maxlen=max_len,padding='post')
vocab_size=len(tokenizer.word_index)+1

In [131]:
len(tokenizer.word_index)

43465

In [146]:
type(X_t)

numpy.ndarray

In [160]:
X_t.shape

(96778, 40)

In [149]:
# create a weight matrix for words in training docs
embedding_matrix = zeros((vocab_size, 100))
for word, i in tokenizer.word_index.items():
  embedding_vector = embeddings_index.get(word)
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector

In [150]:
embedding_matrix[:5]

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+0

# Creat Model

In [151]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding

In [152]:
model=Sequential()
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=40, trainable=False)
model.add(e)
model.add(Flatten())
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [153]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [154]:
model.summary()

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_8 (Embedding)              │ ?                           │       4,346,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_7 (Flatten)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,346,600 (16.58 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 4,346,600 (16.58 MB)

In [157]:
model.fit(X_t, Y_train, validation_data=(X_v,Y_test), epochs=10)

Epoch 1/10
3025/3025 ━━━━━━━━━━━━━━━━━━━━ 30s 10ms/step - accuracy: 0.7588 - loss: 0.4969 - val_accuracy: 0.7467 - val_loss: 0.5179
Epoch 2/10
3025/3025 ━━━━━━━━━━━━━━━━━━━━ 46s 12ms/step - accuracy: 0.7686 - loss: 0.4805 - val_accuracy: 0.7476 - val_loss: 0.5253
Epoch 3/10
3025/3025 ━━━━━━━━━━━━━━━━━━━━ 39s 13ms/step - accuracy: 0.7785 - loss: 0.4654 - val_accuracy: 0.7510 - val_loss: 0.5194
Epoch 4/10
3025/3025 ━━━━━━━━━━━━━━━━━━━━ 39s 13ms/step - accuracy: 0.7846 - loss: 0.4513 - val_accuracy: 0.7501 - val_loss: 0.5231
Epoch 5/10
3025/3025 ━━━━━━━━━━━━━━━━━━━━ 32s 10ms/step - accuracy: 0.7923 - loss: 0.4397 - val_accuracy: 0.7440 - val_loss: 0.5381
Epoch 6/10
3025/3025 ━━━━━━━━━━━━━━━━━━━━ 47s 13ms/step - accuracy: 0.7971 - loss: 0.4266 - val_accuracy: 0.7513 - val_loss: 0.5350
Epoch 7/10
3025/3025 ━━━━━━━━━━━━━━━━━━━━ 27s 9ms/step - accuracy: 0.7989 - loss: 0.4240 - val_accuracy: 0.7546 - val_loss: 0.5486
Epoch 8/10
3025/3025 ━━━━━━━━━━━━━━━━━━━━ 48s 11ms/step - accuracy: 0.8038 - 

# Do not Use Pre-Trained Embedding

In [158]:
encoded_docs_train = [one_hot(d, vocab_size) for d in X_train]
encoded_docs_test = [one_hot(d, vocab_size) for d in X_test]


In [161]:
len(encoded_docs_test)

64519

In [162]:
max_length=40
padded_docs_train = pad_sequences(encoded_docs_train, maxlen=max_length, padding='post')
padded_docs_test = pad_sequences(encoded_docs_test, maxlen=max_length, padding='post')

In [163]:
padded_docs_train.shape

(96778, 40)

In [164]:
Y_train.shape

(96778,)

In [165]:

model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=max_length))
model.add(Flatten())
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# summarize the model
print(model.summary())


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_9 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_8 (Flatten)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_15 (Dense)                     │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [ ]:
# fit the model
model.fit(padded_docs_train, Y_train,validation_data=(padded_docs_test,Y_test), epochs=10)


Epoch 1/10
3025/3025 ━━━━━━━━━━━━━━━━━━━━ 254s 83ms/step - accuracy: 0.7731 - loss: 0.4753 - val_accuracy: 0.8303 - val_loss: 0.3819
Epoch 2/10
3025/3025 ━━━━━━━━━━━━━━━━━━━━ 257s 85ms/step - accuracy: 0.9031 - loss: 0.2389 - val_accuracy: 0.8426 - val_loss: 0.3870
Epoch 3/10
3025/3025 ━━━━━━━━━━━━━━━━━━━━ 247s 80ms/step - accuracy: 0.9709 - loss: 0.0831 - val_accuracy: 0.8414 - val_loss: 0.5380
Epoch 4/10
3025/3025 ━━━━━━━━━━━━━━━━━━━━ 253s 77ms/step - accuracy: 0.9881 - loss: 0.0349 - val_accuracy: 0.8447 - val_loss: 0.7970
Epoch 5/10
3025/3025 ━━━━━━━━━━━━━━━━━━━━ 272s 80ms/step - accuracy: 0.9927 - loss: 0.0214 - val_accuracy: 0.8414 - val_loss: 0.9118
Epoch 6/10
3025/3025 ━━━━━━━━━━━━━━━━━━━━ 247s 82ms/step - accuracy: 0.9944 - loss: 0.0163 - val_accuracy: 0.8425 - val_loss: 1.0492
Epoch 7/10
3025/3025 ━━━━━━━━━━━━━━━━━━━━ 241s 80ms/step - accuracy: 0.9967 - loss: 0.0104 - val_accuracy: 0.8396 - val_loss: 1.1792
Epoch 8/10
 914/3025 ━━━━━━━━━━━━━━━━━━━━ 2:50 81ms/step - accuracy: 